# 23 overfitting and underfitting

### underfitting: estimated<Ground-truth
<img src='./images/6.png' style="zoom:50%">

### overfitting: Ground-truth<Estimated
<img src='./images/7.png' style="zoom:50%">
<mark>generalization performance decrease<mark>

## train test trade-off
<img src='./images/8.png' style="zoom:50%">

## k-fold cross validation

## Train-Validation-Test

In [1]:
import  torch
import  torch.nn as nn
import  torch.nn.functional as F
import  torch.optim as optim
from    torchvision import datasets, transforms

In [2]:
batch_size=200
learning_rate=0.01
epochs=10

In [5]:
train_db = datasets.MNIST('./data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))
train_loader = torch.utils.data.DataLoader(
    train_db,
    batch_size=batch_size, shuffle=True)

test_db = datasets.MNIST('./data', train=False, transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
]))
test_loader = torch.utils.data.DataLoader(test_db,
    batch_size=batch_size, shuffle=True)

In [6]:
print('train:', len(train_db), 'test:', len(test_db))
train_db, val_db = torch.utils.data.random_split(train_db, [50000, 10000])
print('db1:', len(train_db), 'db2:', len(val_db))
train_loader = torch.utils.data.DataLoader(
    train_db,
    batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(
    val_db,
    batch_size=batch_size, shuffle=True)


train: 60000 test: 10000
db1: 50000 db2: 10000


In [7]:
class MLP(nn.Module):

    def __init__(self):
        super(MLP, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(784, 200),
            nn.LeakyReLU(inplace=True),
            nn.Linear(200, 200),
            nn.LeakyReLU(inplace=True),
            nn.Linear(200, 10),
            nn.LeakyReLU(inplace=True),
        )

    def forward(self, x):
        x = self.model(x)

        return x

In [8]:
device = torch.device('cuda:0')
net = MLP().to(device)
optimizer = optim.SGD(net.parameters(), lr=learning_rate)
criteon = nn.CrossEntropyLoss().to(device)

In [9]:
for epoch in range(epochs):

    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.view(-1, 28*28)
        data, target = data.to(device), target.cuda()

        logits = net(data)
        loss = criteon(logits, target)

        optimizer.zero_grad()
        loss.backward()
        # print(w1.grad.norm(), w2.grad.norm())
        optimizer.step()

        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))


    test_loss = 0
    correct = 0
    for data, target in val_loader:
        data = data.view(-1, 28 * 28)
        data, target = data.to(device), target.cuda()
        logits = net(data)
        test_loss += criteon(logits, target).item()

        pred = logits.data.max(1)[1]
        correct += pred.eq(target.data).sum()

    test_loss /= len(val_loader.dataset)
    print('\nVAL set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(val_loader.dataset),
        100. * correct / len(val_loader.dataset)))

Train Epoch: 0 [0/50000 (0%)]	Loss: 2.292923
Train Epoch: 0 [20000/50000 (40%)]	Loss: 2.129498
Train Epoch: 0 [40000/50000 (80%)]	Loss: 1.629001

VAL set: Average loss: 0.0071, Accuracy: 6388/10000 (63%)

Train Epoch: 1 [0/50000 (0%)]	Loss: 1.583038
Train Epoch: 1 [20000/50000 (40%)]	Loss: 1.112609
Train Epoch: 1 [40000/50000 (80%)]	Loss: 1.112436

VAL set: Average loss: 0.0046, Accuracy: 7461/10000 (74%)

Train Epoch: 2 [0/50000 (0%)]	Loss: 0.962306
Train Epoch: 2 [20000/50000 (40%)]	Loss: 0.650866
Train Epoch: 2 [40000/50000 (80%)]	Loss: 0.694571

VAL set: Average loss: 0.0032, Accuracy: 8097/10000 (80%)

Train Epoch: 3 [0/50000 (0%)]	Loss: 0.645702
Train Epoch: 3 [20000/50000 (40%)]	Loss: 0.474482
Train Epoch: 3 [40000/50000 (80%)]	Loss: 0.452642

VAL set: Average loss: 0.0019, Accuracy: 8934/10000 (89%)

Train Epoch: 4 [0/50000 (0%)]	Loss: 0.319428
Train Epoch: 4 [20000/50000 (40%)]	Loss: 0.376218
Train Epoch: 4 [40000/50000 (80%)]	Loss: 0.351451

VAL set: Average loss: 0.0017, Acc

In [10]:
test_loss = 0
correct = 0
for data, target in test_loader:
    data = data.view(-1, 28 * 28)
    data, target = data.to(device), target.cuda()
    logits = net(data)
    test_loss += criteon(logits, target).item()

    pred = logits.data.max(1)[1]
    correct += pred.eq(target.data).sum()

test_loss /= len(test_loader.dataset)
print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))


Test set: Average loss: 0.0012, Accuracy: 9301/10000 (93%)

